In [1]:
import sys
#!{sys.executable} -m pip install dicomsdl
!pip install /kaggle/input/rsnawheelspackages/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#import pydicom as dicom
import dicomsdl as dicom
import tensorflow as tf
import tensorflow_io as tfio
from PIL import Image as im
import os
from tqdm import tqdm
import cv2
from PIL import Image
from joblib import Parallel, delayed
import gc
import random

Processing /kaggle/input/rsnawheelspackages/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl


## Enter the path of the test dataset and image dataset here

In [2]:

datasetPath = '/kaggle/input/rsna-breast-cancer-detection/test.csv'
imgPath = '/kaggle/input/rsna-breast-cancer-detection/test_images/'


## Data processing pipeline

In [3]:



def crop(sideName, imgName):
    if sideName == 'L':
        colind=[]
        for r,row in enumerate(imgName):
            for c,col in enumerate(row):
                if col==0:
                    colind.append(c)
                    break
        crop_size = max(colind)
        imgName = imgName[0:512,0:crop_size]
        imgName = cv2.resize(imgName,(128,128))
        
    if sideName == 'R':
        colind=[]
        for r,row in enumerate(imgName):
            for c,col in enumerate(row):
                if col!=0:
                    colind.append(c)
                    break
        crop_size = min(colind)
        imgName = imgName[0:512,crop_size:512]
        imgName = cv2.resize(imgName,(128,128))
    
    return imgName    

    gc.collect()
    
def crop_reverse(sideName, imgName):
    if sideName == 'R':
        colind=[]
        for r,row in enumerate(imgName):
            for c,col in enumerate(row):
                if col==0:
                    colind.append(c)
                    break
        crop_size = max(colind)
        imgName = imgName[0:512,0:crop_size]
        imgName = cv2.resize(imgName,(128,128))
        
    if sideName == 'L':
        colind=[]
        for r,row in enumerate(imgName):
            for c,col in enumerate(row):
                if col!=0:
                    colind.append(c)
                    break
        crop_size = min(colind)
        imgName = imgName[0:512,crop_size:512]
        imgName = cv2.resize(imgName,(128,128))
    
    return imgName    

    gc.collect()
    

def img_process(i,filename,sides):
    
    #ds = dicom.dcmread(filename)
    ds = dicom.open(filename)
    ds = ds.pixelData()
    ds = cv2.normalize(ds, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    ds = cv2.resize(ds,(512,512))
    ds = np.where(ds >= 0.9999, 0,ds)
    
    try:
        ds = np.array(crop(sides[i], ds))   
    except:
        ds = np.array(crop_reverse(sides[i], ds))
    return ds 

    #train_data.loc[i,'img_data'] = [img_fin]
    #train_data.to_csv('/kaggle/working/training_img_data.csv') 
    
    gc.collect()

def dcmToPix(datasetPath, imgPath):
    dataset = pd.read_csv(datasetPath)
    dataset=dataset[:50]
    
    
    patient_ids = dataset['patient_id']
    image_ids = dataset['image_id']
    sides  = dataset['laterality']

    imgData = []

    for pi, ii, leng in zip(patient_ids, image_ids, range(len(patient_ids))):
        imgData.append(imgPath + str(pi) + '/' + str(ii) + '.dcm')

    dataset['img_data'] = " "
    
    result = Parallel(n_jobs=128)(\
    delayed(img_process)(i, fname, sides) for i, fname in zip(range(len(imgData)),tqdm(imgData))\
    )
    
    dataset['img_data'] = result
    dataset.to_pickle('imgData.pkl' )
    
    return result
    
def random_rotate(imgData):
    imgData = imgData.reshape(128,128)
    imgData = im.fromarray(imgData)
    imgData = np.asarray(imgData.rotate(random.randint(0, 360)))
    return imgData

## Implementing the data processing pipeline to the data

In [4]:
testImgData = dcmToPix(datasetPath, imgPath)

 75%|███████▌  | 3/4 [00:00<00:00, 94.75it/s]


## Normalizing the data and processing the data to make it compatible with the model

In [5]:

imgDataFrame = {'img_data':testImgData}
imgData2 = pd.DataFrame(imgDataFrame)
testImgData=imgData2


In [6]:
imgDataList=[]
for j in tqdm(testImgData['img_data']):
    imgDataList.append(random_rotate(j))
testImgData['img_data'] = imgDataList

100%|██████████| 4/4 [00:00<00:00, 1171.02it/s]


In [7]:
test=testImgData


test_features=[]
for i in test['img_data']:
    test_features.append([i])
test_features=np.array(test_features)


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
norm_features= []
for i in range(len(test_features)):
    norm_features.append(scaler.fit_transform(test_features[i][0]))
test_features=np.array(norm_features)

test_features = test_features.reshape(len(test_features),128,128,1)



## Loading the trained model

I have already implemented the convolutional neural network model to the train dataset. Because it took me so long to run the notebook for the model training, I am not uploading the previous notebooks. It is necessary to run above cells to run the model and predict the outcomes.

**Please put the path of the trained model. The model is saved in the dataset named "pre trained model of breast cancer" under the name of Ridhesh Goti**

In [8]:
from tensorflow import keras
model = keras.models.load_model("/kaggle/input/pre-trained-model-of-breast-cancer/trained_model_breast_cancer.h5")

2023-01-20 11:42:03.194717: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## Predictions

In [9]:
pred = model.predict(test_features)
bin_pred = []
for i in pred:
    if i>=0.5:
        bin_pred.append(1)
    else:
        bin_pred.append(0)
bin_pred = np.array(bin_pred)


2023-01-20 11:42:03.934243: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


## Saving the model to the current directory as submission.csv

In [10]:
testData = pd.read_csv(datasetPath)

submissionFrame={'prediction_id':testData['prediction_id'],'cancer':bin_pred}
submission = pd.DataFrame(submissionFrame)
submission.to_csv('submission.csv',index=False)

Please let me know if there is anything missing.